In [12]:
from qdrant_client import QdrantClient
from qdrant_openapi_client.models import VectorParams, Distance
import numpy as np

client = QdrantClient(host="localhost", port=6333, grpc_port=6334, prefer_grpc=True)

In [13]:
def collection_p(collection: str) -> bool:
    return client.collection_exists(collection_name=collection)

COLL = "developer-docs"

print(collection_p(COLL))

True


I0000 00:00:1722124241.548019   34929 config.cc:230] gRPC experiments enabled: call_status_override_on_cancellation, event_engine_dns, event_engine_listener, http2_stats_fix, monitoring_experiment, pick_first_new, trace_record_callops, work_serializer_clears_time_cache


In [26]:
from qdrant_client.http.models.models import CollectionInfo

def collection_info(collection: str) -> dict:
    return client.get_collection(collection).model_dump()

def collection_info_status(collection: str):
    return collection_info(collection).get("status")

def collection_info_points_count(collection: str):
    return collection_info(collection).get("points_count")

def collection_info_config(collection: str):
    return collection_info(collection).get("config")

collection_info_config(COLL)

{'params': {'vectors': {'size': 384,
   'distance': <Distance.COSINE: 'Cosine'>,
   'hnsw_config': None,
   'quantization_config': None,
   'on_disk': None,
   'datatype': None,
   'multivector_config': None},
  'shard_number': 1,
  'sharding_method': None,
  'replication_factor': 1,
  'write_consistency_factor': 1,
  'read_fan_out_factor': None,
  'on_disk_payload': True,
  'sparse_vectors': None},
 'hnsw_config': {'m': 16,
  'ef_construct': 100,
  'full_scan_threshold': 10000,
  'max_indexing_threads': 0,
  'on_disk': False,
  'payload_m': None},
 'optimizer_config': {'deleted_threshold': 0.2,
  'vacuum_min_vector_number': 1000,
  'default_segment_number': 0,
  'max_segment_size': None,
  'memmap_threshold': None,
  'indexing_threshold': 10000,
  'flush_interval_sec': 5,
  'max_optimization_threads': 1},
 'wal_config': {'wal_capacity_mb': 32, 'wal_segments_ahead': 0},
 'quantization_config': None}

In [35]:
from qdrant_client.models import Filter, FieldCondition, Range

In [44]:
from sentence_transformers import SentenceTransformer


def search(collection: str, text: str, limit: int = 5):
    encoder = SentenceTransformer("all-MiniLM-L6-v2")  # Model to create embeddings
    hits = client.search(
        collection_name=collection,
        query_vector=encoder.encode(text).tolist(),
        limit=limit,
    )
    return [hit.payload for hit in hits]

In [45]:
search(COLL, "python")

[{'text': 'Practical Implementation in Python',
  'sections': ['articles', 'articles/sparse-vectors'],
  'titles': ['Sparse Vectors in Qdrant: Pure Vector-based Hybrid Search - Qdrant'],
  'tag': 'h2',
  'location': 'html > body > div:nth-of-type(1) > div:nth-of-type(1) > div > section > article > h2:nth-of-type(9)',
  'url': '/articles/sparse-vectors/'},
 {'text': 'But there are two main reasons for us to use Python for this:',
  'sections': ['benchmarks', 'benchmarks/benchmark-faq'],
  'titles': ['Benchmarks F.A.Q. - Qdrant',
   'Why we decided to test with the Python client'],
  'tag': 'p',
  'location': 'html > body > div:nth-of-type(1) > div > div > div > section > article > p:nth-of-type(10)',
  'url': '/benchmarks/benchmark-faq/'},
 {'text': 'Get the Python client if you’re familiar with Python.',
  'sections': ['documentation'],
  'titles': ['Qdrant Documentation - Qdrant', 'First-Time Users:'],
  'tag': 'li',
  'location': 'html > body > div:nth-of-type(1) > section > div > di